# THIS IS THE NOTEBOOK FOR FINE-TUNING THE PRE-TRAINED BART MODEL

In [14]:
MODEL_PATH = "models/bart_large_xsum_samsum"
MODEL_NAME = "facebook/bart-large-xsum"
CHECKPOINT = "checkpoint-5500"
PRETRAIN_DATASET = "samsum"
METRIC = "rouge"

In [2]:
#IMPORTING LIBRARIES AND THE DATASET
import os
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric
import nltk
nltk.download('punkt')
import numpy as np

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
raw_dataset = load_dataset(PRETRAIN_DATASET)
metric = load_metric(METRIC)

/home/kristyna/Documents/Study/MFF/2_semestr/npfl087/automin/AutoMin2023/experiments/automin2021/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/kristyna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Found cached dataset samsum (/home/kristyna/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)
100%|██████████| 3/3 [00:00<00:00, 183.62it/s]
/tmp/ipykernel_7086/4147100220.py:11: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric(METRIC)


In [3]:
#PREPROCESSING THE DATA

max_input_length = 512
max_target_length = 128

def preprocess_data(examples):
    inputs = [doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [4]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [5]:
tokenized_dataset = raw_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/kristyna/Documents/Study/MFF/2_semestr/npfl087/automin/AutoMin2023/experiments/automin2021/venv/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [6]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [7]:
#FINE-TUNING THE MODEL

from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [9]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    MODEL_PATH,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
#TRAINING AND EVALUATION

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [12]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.181200,1.489181,53.792600,29.374100,44.720900,49.509800,27.036700
2,0.943100,1.494375,54.490100,29.110200,44.478500,50.051200,31.474300
2,0.724700,1.586577,54.340200,28.945400,44.613100,49.735800,30.374100


TrainOutput(global_step=5523, training_loss=0.915807216685038, metrics={'train_runtime': 4620.8235, 'train_samples_per_second': 9.565, 'train_steps_per_second': 1.195, 'total_flos': 2.609313301001011e+16, 'train_loss': 0.915807216685038, 'epoch': 3.0})

In [15]:
################################################################################################
########## SAVE THE MODEL AND USE IT IN THE FORM OF A PIPEPLINE AS DONE IN THIS CELL ##########
################################################################################################

from transformers import pipeline

summarizer = pipeline("summarization", model=os.path.join(MODEL_PATH, CHECKPOINT))


conversation = '''Kartik: We want the model to be able to capture the significant details from a conversation thread.
Nidhir: What should we do in order to achieve that? We can't expect it to do topical segmentaion. It's a pretty difficult task.
Aakash: Yes, moreover, we might encounter long-stretched topics in the transcript.
Kartik: I'm planning to segment the transcript on the basis of token length. We should find a sweet spot at which the model doesn't have to choose between multiple topics, at the same time, extract the relevant info from it.
Aakash: Sounds promising.
Nidhir: Hey, what happened to the model that we trained?
Aakash: I think, it would've finished evaluating on the data.
Kartik: If it's done evaluating, send me the performances ASAP.
Nidhir: Okay, sure.
Kartik: And yes, don't forget to upload those on this link - https://github.com/cruxieu17/automin-2021-submission 🙂.
Aakash: Alright, consider it done.
Kartik: K then, Bye!
Nidhir: Bye bye.                                      
'''
summary = summarizer(conversation)
summary = summary[0]['summary_text']

print(summary)

Nidhir and Aakash are training a machine learning model. Kartik is planning to segment the transcript on the basis of token length.
